<a href="https://colab.research.google.com/github/zacharylazzara/tent-detection/blob/main/Tent_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Referenced Materials**

* https://amaarora.github.io/2020/09/13/unet.html
* https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5
* https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
* https://www.youtube.com/watch?v=IHq1t7NxS8k

The majority of the UNet implementation comes from the referenced YouTube video

In [ ]:
WORK_IN_DRIVE = True # Work entirely in Google Drive or not

# Imports and Initialization

In [ ]:
# Imports
%cd /content
from google.colab import drive
from google.colab import files
!mkdir -p drive/
drive.mount('drive/')

!pip install segmentation-models-pytorch -q
!pip install -U albumentations -q

import math
import sys
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import csv
import random
import matplotlib.patches as mpatches
import torch
import torchvision
import torch.utils.data
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms.functional as TF
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
import cv2
# from google.colab.patches import cv2_imshow
from pathlib import Path
from glob import glob
import pickle
import shutil

/content
Mounted at drive/
     |████████████████████████████████| 102 kB 10.6 MB/s 
     |████████████████████████████████| 376 kB 39.8 MB/s 
     |████████████████████████████████| 58 kB 7.5 MB/s 
     |████████████████████████████████| 123 kB 35.6 MB/s 


In [ ]:
# Initialize Environment
%env DRIVE_DIR      = drive/MyDrive/Thesis/content/
%env SRC_DIR        = sarpol-zahab-tents/
%env DATA_DIR       = data/
%env TRAINING_DIR   = data/training/
%env VALIDATION_DIR = data/validation/
%env TRAIN_IMG_DIR  = data/training/features/
%env TRAIN_LBL_DIR  = data/training/targets/
%env VAL_IMG_DIR    = data/validation/features/
%env VAL_LBL_DIR    = data/validation/targets/
%env OUTPUT_DIR     = output/

DRIVE_DIR           = os.environ.get("DRIVE_DIR")
SRC_DIR             = os.environ.get("SRC_DIR")
DATA_DIR            = os.environ.get("DATA_DIR")
TRAINING_DIR        = os.environ.get("TRAINING_DIR")
VALIDATION_DIR      = os.environ.get("VALIDATION_DIR")
TRAIN_IMG_DIR       = os.environ.get("TRAIN_IMG_DIR")
TRAIN_LBL_DIR       = os.environ.get("TRAIN_LBL_DIR")
VAL_IMG_DIR         = os.environ.get("VAL_IMG_DIR")
VAL_LBL_DIR         = os.environ.get("VAL_LBL_DIR")
OUTPUT_DIR          = os.environ.get("OUTPUT_DIR")

env: DRIVE_DIR=drive/MyDrive/Thesis/content/
env: SRC_DIR=sarpol-zahab-tents/
env: DATA_DIR=data/
env: TRAINING_DIR=data/training/
env: VALIDATION_DIR=data/validation/
env: TRAIN_IMG_DIR=data/training/features/
env: TRAIN_LBL_DIR=data/training/targets/
env: VAL_IMG_DIR=data/validation/features/
env: VAL_LBL_DIR=data/validation/targets/
env: OUTPUT_DIR=output/


In [ ]:
if WORK_IN_DRIVE:
  os.chdir(DRIVE_DIR)

In [ ]:
# Initialize Directories
%%bash
echo "Working in Directory: $(pwd)"

if [ -d 'sample_data' ]; then
  rm -r sample_data
fi

if [ ! -d $SRC_DIR ] && [ ! -d $DATA_DIR ]; then
  git clone https://github.com/tofighi/sarpol-zahab-tents.git
fi

if [ ! -d $DATA_DIR ]; then
  mkdir -p $DATA_DIR
  cp $SRC_DIR/data/sarpol_counts.csv $DATA_DIR
fi

if [ ! -d $TRAIN_IMG_DIR ]; then
  mkdir -p $TRAIN_IMG_DIR
fi

if [ ! -d $TRAIN_LBL_DIR ]; then
  mkdir -p $TRAIN_LBL_DIR
fi

if [ ! -d $VAL_IMG_DIR ]; then
  mkdir -p $VAL_IMG_DIR
fi

if [ ! -d $VAL_LBL_DIR ]; then
  mkdir -p $VAL_LBL_DIR
fi

Working in Directory: /content/drive/MyDrive/Thesis/content


# Configuration

In [ ]:
# Configuration
MODEL_ON_DRIVE        = True
SAVE_OUTPUT_TO_DRIVE  = True
SAVE_DATASET_TO_DRIVE = True

DATA_AUGMENTATION     = False

# Dataset
TEST_SIZE = 0.3
RANDOM_STATE = 123
ALLOW_IRRELEVANT = True # If images don't have tents do we want to throw them out or not?
SARPOL = False # do we want to download the very large Sarpol image?

TILE = True # If we want to generate the final big image
EPOCH_TILE = True # Tile during epochs instead of after

GRAYSCALE = False # Print tents as grayscale or not

LIVE_VISUALIZE = False

# TODO: lets just load all data as validation data to generate the map data and such

RAW_IMAGE_DIR = f"{SRC_DIR}data/images/"
RAW_LABEL_DIR = f"{SRC_DIR}data/labels/"

# Overview Directories #
OVERVIEWS           = f"{OUTPUT_DIR}overviews/"
ESTIMATE_OVERVIEWS  = f"{OVERVIEWS}estimates/"
TARGET_OVERVIEWS    = f"{OVERVIEWS}targets/"
FEATURE_OVERVIEWS   = f"{OVERVIEWS}features/"

OVERLAYS            = f"{OVERVIEWS}overlays/"

ESTIMATE_OVERLAYS   = f"{OVERLAYS}estimates/"
TARGET_OVERLAYS     = f"{OVERLAYS}targets/"

HEATMAPS            = f"{OVERVIEWS}heatmaps/"
ESTIMATE_HEATMAPS   = f"{HEATMAPS}estimates/"
TARGET_HEATMAPS     = f"{HEATMAPS}targets/"
########################

# Pickle Outputs #######
PICKLES       = f"{OUTPUT_DIR}pickles/"
RAW_PICKLE    = f"{PICKLES}raw_dataset.pkl"
DATA_PICKLE   = f"{PICKLES}dataset.pkl"
RESULT_PICKLE = f"{PICKLES}results.pkl"

RESULT_CSV    = f"{PICKLES}results.csv"
########################

TENT_CSV      = f"{DATA_DIR}sarpol_counts.csv"
LOAD_FROM_CSV = True

BLOB_LOCALIZATION = False
KMEANS_LOCALIZATION = False # Causes error in kmeans localization when saving training data?

# Display Limit
DISP_LIMIT    = 1 # Maximum number of images to display
DISP_RESULTS  = True
DISP_SCALE    = 2#150 # Amount to integer divide displayed figure scale by (set to 1 to disable); useful if the notebook keeps crashing

# Images
IMG_FORMAT    = "png"
BRIGHTNESS    = 0.5 # set to 1 for no dimming

# Checkpoints
CHECKPOINT    = "checkpoint.pt"
if MODEL_ON_DRIVE and not WORK_IN_DRIVE:
  CHECKPOINT = f"{DRIVE_DIR}{CHECKPOINT}"


print(f"Is CUDA available? {torch.cuda.is_available()}")
# Hyperparameters
LEARNING_RATE = 1e-4 # 1x10^-4 = 0.0001
DEVICE        = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE    = 1#5 # Batch size defines the prediction batch; set to 1 if we want individual files
NUM_EPOCHS    = 1 #50 #100
NUM_WORKERS   = 2
IMAGE_HEIGHT  = 512
IMAGE_WIDTH   = 512
PIN_MEMORY    = True
LOAD_MODEL    = True

# Convolution Settings
C_KERNEL      = 3     # This is the matrix that slides across the image (we define matrix size, so kernel = 3 means 3x3 matrix that slides across the image)
C_STRIDE      = 1     # Number of pixels the kernel slides over the input (how many pixels we move the filter at a time)
C_PADDING     = 1     # Sometimes the filter doesn't perfectly fit the input image, in which case we can pad with 0s or drop the part of the image that didn't fit (called valid padding)
C_BIAS        = False # Bias is false in this case because we're using BatchNorm2d (bias would be canceled by the batch norm, so we set it to false)
R_INPLACE     = True  #

# UNet Settings
IN_CHANNELS   = 3
OUT_CHANNELS  = 1 # We're doing binary image segmentation (because our masks are black and white), so we can output a single channel
U_FEATURES    = [64, 128, 256, 512] # Features come from the architecture (the number above the boxes)

# UNet Pool Settings
P_KERNEL      = 2
P_STRIDE      = 2

# Final layer kernel size
F_KERNEL      = 1 # Because we're outputting the final image here



# Epsilon is for the accuracy, so we don't get division by 0. Using 1, because working with integers.
EPSILON = 1 #sys.float_info.epsilon



# Define and create directory structures
def dirs(root=OUTPUT_DIR):
  datasets = ["validation", "training"]
  dirs = {}
  for dataset in datasets:
    data_dir = VALIDATION_DIR if dataset == 'validation' else TRAINING_DIR
    dataset_dir = f"{root}{dataset}/"
    dirs[dataset] = {"features":       f"{data_dir}features/",
                     "targets":        f"{data_dir}targets/",
                     "estimates":      f"{dataset_dir}estimates/"}
  return dirs
DIRECTORIES = dirs()
for dataset in DIRECTORIES.values():
  for dir in dataset.values():
    if not os.path.exists(dir):
      os.makedirs(dir)

Is CUDA available? True


In [ ]:
def load(path):
  imgs = []
  for filepath in sorted(glob(path)):
    with Image.open(filepath) as img:
      imgs.append((np.asarray(img), os.path.basename(path).split(f".", 1)[0]))
  return imgs #np.array([(np.asarray(Image.open(path)), os.path.basename(path).split(f".", 1)[0]) for path in sorted(glob(path))])
  
def load_filenames(path):
  return np.array([os.path.basename(path).split(f".", 1) for path in sorted(glob(path))])

MAP = None
if SARPOL:
  !gdown --id 1-YUbFjwFL2G5r8TudKS0XK56BjJ0KsTK
  MAP = load("sarpol.png")[0][0]
  print(f"Sarpol Shape: {MAP.shape}\n")

# if LOAD_MODEL:
# TODO: if we don't have the checkpoint already, then download it; otherwise use the one we have
#   !gdown --id 1ulHrgSyNwo1X2WYmQyeb4lIxIIJ4Xs-G

# Preprocessing

In [ ]:
# Preparing Data
def find_square_coordinates(record, row = 0, max_records = 256):
  row_length = int(math.sqrt(max_records))
  if record >= row_length:
    return find_square_coordinates(record - row_length, row + 1, max_records)
  else:
    return (row, record)

def percent(sample, total):
  return f"({sample}/{total}) = {((sample/total)*100):.2f}%"

def populate_dirs(dataset):
  print("\nPopulating training and validation directories...")

  # TODO: this should delete everything in the dirs before running, to ensure runs are identical

  data_dir = {"training":{"img":TRAIN_IMG_DIR, "lbl":TRAIN_LBL_DIR}, "validation":{"img":VAL_IMG_DIR, "lbl":VAL_LBL_DIR}}

  data_loop = tqdm(dataset)
  for subset in data_loop:
    for record in dataset[subset]["x"]:
      for dir in record["dir"]:
        input = f"{record['dir'][dir]}{record['id']}.{record['format']}"
        output = f"{data_dir[subset][dir]}{record['id']}.{IMG_FORMAT}"
        data_loop.set_description(f"Input: {input}, Output: {output}")
        with Image.open(input) as img:
          img.save(output)
  print("\n\nDone.")

raw_dataset = {"x":[], "y":[]}
dataset = {"training":{"x":[], "y":[]}, "validation":{"x":[], "y":[]}}
if os.path.exists(PICKLES):
  print("Loading pickles...")
  with open(RAW_PICKLE, "rb") as raw_pickle:
    raw_dataset = pickle.load(raw_pickle)
  with open(DATA_PICKLE, "rb") as data_pickle:
    dataset = pickle.load(data_pickle)
  print("Done.")
else:
  src_imgs = load_filenames(f"{RAW_IMAGE_DIR}*")
  src_lbls = load_filenames(f"{RAW_LABEL_DIR}*")
  if src_imgs.shape[0] == src_lbls.shape[0]:
    n = src_lbls.shape[0]

    with open(TENT_CSV, newline='') as csvfile:
      raw_dataset["y"] = list(csv.reader(csvfile))
    for index in range(n):
      if src_imgs[index][0] == src_lbls[index][0]:
        id = src_imgs[index][0]
        format = src_imgs[index][1]

        raw_dataset["x"].append({
          "dir":{
            "img":f"{RAW_IMAGE_DIR}",
            "lbl":f"{RAW_LABEL_DIR}"
          },
          "id":f"{id}",
          "format":f"{format}",
          "tile_coordinates":find_square_coordinates(index)
        })
      else:
        print("ERROR: ID mismatch!")
    os.makedirs(PICKLES)
    with open(RAW_PICKLE, "wb") as output:
      pickle.dump(raw_dataset, output, pickle.HIGHEST_PROTOCOL)
  else:
    print("ERROR: Shape mismatch!")
  dataset["training"]["x"], dataset["validation"]["x"], dataset["training"]["y"], dataset["validation"]["y"] = train_test_split(raw_dataset["x"], raw_dataset["y"], test_size=TEST_SIZE, random_state=RANDOM_STATE)
  with open(DATA_PICKLE, "wb") as output:
    pickle.dump(dataset, output, pickle.HIGHEST_PROTOCOL)

irrelevant = raw_dataset["y"].count(0)
total = len(raw_dataset["x"])
relevant = total - irrelevant
print(f"\nRaw Dataset Size: {total}\nRelevant Percentage: {percent(relevant, total)}, Irrelevant Percentage: {percent(irrelevant, total)}")
print(f"Allow Irrelevant Data? {'YES' if ALLOW_IRRELEVANT else 'NO'}")

training_data = len(dataset["training"]["x"])
validation_data = len(dataset["validation"]["x"])
total_usable = training_data + validation_data
print(f"\nTraining Percentage: {percent(training_data, total_usable)}, Validation Percent: {percent(validation_data, total_usable)}")

if os.path.exists(SRC_DIR):
  populate_dirs(dataset)
elif os.path.exists(DATA_DIR):
  print(f"\nData directory '{DATA_DIR}' has already been populated in a previous run.")
else:
  print(f"\nSource directory '{SRC_DIR}' not found, unable to populate '{DATA_DIR}'!")

Loading pickles...
Done.

Raw Dataset Size: 256
Relevant Percentage: (256/256) = 100.00%, Irrelevant Percentage: (0/256) = 0.00%
Allow Irrelevant Data? YES

Training Percentage: (179/256) = 69.92%, Validation Percent: (77/256) = 30.08%

Data directory 'data/' has already been populated in a previous run.


In [ ]:
# Cleanup
%%bash
if [ -d $SRC_DIR ]; then
  rm -r $SRC_DIR
fi

# UNet

**U-Net Architecture:**

The first half of the architecture is the down-sampling process. At each stage, two 3x3 convolutions are used. The output is then pooled and becomes the input for the next 3x3 convolution and so on, until the bottom of this diagram is reached.

The second half of the architecture is the up-sampling process, which is a reflection of the down-sampling process.

In [ ]:
# UNet Model
# Adapted From https://www.youtube.com/watch?v=IHq1t7NxS8k
# Referenced https://medium.com/@RaghavPrabhu/understanding-of-convolutional-neural-network-cnn-deep-learning-99760835f148

class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DoubleConv, self).__init__()
    self.conv = nn.Sequential (
        nn.Conv2d(in_channels, out_channels, C_KERNEL, C_STRIDE, C_PADDING, bias=C_BIAS), # This is a same convolution (input height*width = output height*width)
        nn.BatchNorm2d(out_channels), # BatchNorm accelerates training by normalizing inputs by re-centering and re-scaling
        nn.ReLU(inplace=R_INPLACE), # ReLU is Rectified Linear Unit; essentially it makes it so negative inputs are discarded and positive inputs are passed through

        # Now we do this a second time but with out_channels to out_channels
        nn.Conv2d(out_channels, out_channels, C_KERNEL, C_STRIDE, C_PADDING, bias=C_BIAS),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=R_INPLACE),
    )

  def forward(self, x):
    return self.conv(x)

# DoubleConv is everything in the first node of the architecture (before pooling)

class UNet(nn.Module):
  def __init__(self, in_channels=3, out_channels=1, features=U_FEATURES):
    super(UNet, self).__init__()
    self.ups = nn.ModuleList()
    self.downs = nn.ModuleList() # Stores the convolutional layers; it's a list, but using ModuleList lets us use BatchNorm2d
    self.pool = nn.MaxPool2d(kernel_size=P_KERNEL, stride=P_STRIDE) # Pooling layer will be used inbetween, in forwarding method
    # Note that the pooling layer will require out inputs to be perfectly divisible by 2 because we're doing a stride of 2
    # Example: 161 x 161 -> MaxPool -> 80 x 80 -> Upsample -> 160 x 160; in this case, we couldn't concatinate the two as they need the same width and height for concat (161x161 is input, 160x160 is output)
    # If input is perfectly dividible by 16 then this issue won't happen (16 because its 4 steps, all dividing by 2 (16/(4*2) = 2))
    # In order to keep the system general, we can either pad the image or crop the image so that it works even if image size isn't perfectly divisible by 16


    # Down-Sampling part of UNet
    for feature in features:
      self.downs.append(DoubleConv(in_channels, feature)) # Mapping some input (in the UNet architecture example, it maps 1 to 64 for the first node)
      in_channels = feature

    # Up-Sampling Part of UNet
    # At 10:20 or so in the video he mentions transposed convoltions that may be a better method to this part, but we'll use similar approach to UNet paper for now
    # We're using the reversed list of features because we're going from the bottom up now
    for feature in reversed(features):
      
      # In_channels is feature*2 here, output is feature
      self.ups.append(
          nn.ConvTranspose2d( 
              feature*2, feature, kernel_size=P_KERNEL, stride=P_STRIDE
          )
      )
      self.ups.append(DoubleConv(feature*2, feature)) # Because we go up then do two convs then go up then do two convs etc

    # Bottom Layer
    self.bottleneck = DoubleConv(features[-1], features[-1]*2) # We're using features[-1] because we want the bottom feature

    # Now we do 1x1 conv which doesn't change height or width just number of channels
    self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=F_KERNEL)
  

  def forward(self, x):
    skip_connections = [] # We skip connections for each stage in the architecture, as we use this part later on
    
    # This loop does all the down-sampling steps until the final step just before the bottom layer (aka bottleneck)
    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = self.pool(x)

    x = self.bottleneck(x)
    skip_connections = skip_connections[::-1] # we wanna go backwards in order when we're doing our concatination; the highest resolution image is the first one; to make things easier, we'll just reverse this list

    # We're using a step of 2 here because we're going up then double conv each iteration
    for i in range(0, len(self.ups), 2):
      x = self.ups[i](x) # doing ConvTranspose2d here
      skip_connection = skip_connections[i//2] # // is integer division; we're doing a step of one ordering here

      # Dealing with images that are not perfectly dividible
      # TODO: perhaps we should look into scaling instead of cropping or padding, or perhaps we should scale before we put the image into the system, so that this is not relevant?
      if x.shape != skip_connection.shape:
        x = TF.resize(x, size=skip_connections.shape[2:]) # we're taking out height and width here; basically, we're resizing the image if it doesn't fit

      concat_skip = torch.cat((skip_connection, x), dim=1) # dim 1 is the channel dimension; we're concationating these things along the channel dimension
      x = self.ups[i+1](concat_skip) # running it through a double conv
    
    return self.final_conv(x)




# Testing the implementation thus far
def test():
  x = torch.randn((3, 1, 160, 160)) # 3 is number of images (batch size), 1 is number of channels, 160 is image height, 160 is image width
  model = UNet(in_channels=1, out_channels=1)
  preds = model(x)
  print(preds.shape)
  print(x.shape)
  assert preds.shape == x.shape

if __name__ == "__main__":
   test()

torch.Size([3, 1, 160, 160])
torch.Size([3, 1, 160, 160])


# Dataset

In [ ]:
# TentDataset
# Adapted From https://www.youtube.com/watch?v=IHq1t7NxS8k

# Data directory should be in the format:
# data
#   train_images
#   train_masks
#   val_images
#   val_masks


# TODO: probably needs to be able to support the case where the target is none?

class TentDataset(Dataset):
  def __init__(self, image_dir=TRAIN_IMG_DIR, mask_dir=TRAIN_LBL_DIR, transform=None):
    self.image_dir = image_dir
    self.mask_dir = mask_dir
    self.transform = transform
    self.images = os.listdir(image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index):
    image_path = os.path.join(self.image_dir, self.images[index])
    mask_path = os.path.join(self.mask_dir, self.images[index])
    image = np.array(Image.open(image_path).convert("RGB"))
    mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32) # We use L since mask is grey scale
    mask[mask == 255.0] = 1.0 # Preprocessing; we change this cuz we're using a sigmoid on the last activation for probability of white pixel, so this makes it work better?

    id = os.path.basename(image_path).replace(f".{IMG_FORMAT}", "")

    if self.transform is not None:
      augmentations = self.transform(image=image, mask=mask)
      image = augmentations["image"]
      mask = augmentations["mask"]

    return image, mask, id

# Utilities

In [ ]:
# Utilities
def save_checkpoint(state, filename=CHECKPOINT):
  print("=> Saving checkpoint\n")
  torch.save(state, filename)

def load_checkpoint(checkpoint, model):
  print("=> Loading checkpoint\n")
  model.load_state_dict(checkpoint["state_dict"])

def get_loaders(train_dir, train_maskdir, val_dir, val_maskdir, batch_size, train_transform, val_transform, num_workers=4, pin_memory=True):
  train_ds = TentDataset(image_dir=train_dir, mask_dir=train_maskdir, transform=train_transform)
  train_loader = DataLoader(train_ds, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=True)
  val_ds = TentDataset(image_dir=val_dir, mask_dir=val_maskdir, transform=val_transform)
  val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=False)
  return train_loader, val_loader

def similarity(estimate, target): # Returns the Jaccard Index of estiamte and target (how similar they are)
  return (EPSILON+(estimate * target).sum()) / (EPSILON+(estimate + target).sum())

def load_image(id, directory=OUTPUT_DIR):
  image = {"record":None, "tile_coord":None, "feature":None, "target":None, "estimate":None, "visualization":None}
  for dirs in DIRECTORIES.values():
    target_path = f"{dirs['targets']}{id}.{IMG_FORMAT}"
    estimate_path = f"{dirs['estimates']}{id}.{IMG_FORMAT}"
    feature_path = f"{dirs['features']}{id}.{IMG_FORMAT}"
    visualization_path = f"{dirs['visualizations']}{id}.{IMG_FORMAT}"

    image["record"] = int(id.split('_')[1])-1
    image["tile_coord"] = find_square_coordinates(image["record"])

    if os.path.exists(target_path):
      with Image.open(target_path).convert("RGBA") as target_img:
        image["target"] = target_img
    
    if os.path.exists(estimate_path):
      with Image.open(estimate_path).convert("RGBA") as estimate_img:
        image["estimate"] = estimate_img
    
    if os.path.exists(feature_path):
      with Image.open(feature_path).convert("RGBA") as feature_img:
        image["feature"] = feature_img
    
    if os.path.exists(visualization_path):
      with Image.open(visualization_path).convert("RGB") as visualization_img:
        image["visualization"] = visualization_img
  return image

def save_heatmap(msk_path, heatmap_output_path, heatmap_format=IMG_FORMAT):
  print(f"\nSaving heatmap to '{heatmap_output_path}'...")
  msk = None
  with Image.open(msk_path).convert("RGB") as mask:
    msk = np.array(mask)
  blur = cv2.GaussianBlur(msk, (55, 55), 0)
  invert = cv2.bitwise_not(blur) # Invert so that colour map works correctly
  heatmap = cv2.applyColorMap(invert, cv2.COLORMAP_JET)
  output = Image.fromarray(heatmap)
  output.save(f"{heatmap_output_path}", format=f"{heatmap_format}")

def save_overlay(img_path, msk_path, overlay_output_path, overlay_format=IMG_FORMAT):
  img = None
  msk = None
  with Image.open(img_path).convert("RGBA") as image:
    img = np.array(image)
  with Image.open(msk_path).convert("RGBA") as mask:
    msk = np.array(mask)

  for channel in range(1, 2):
    msk[msk[:,:,channel] > 0, channel] = 0

  overlay = cv2.addWeighted(img, 1, msk, 1, 0)

  output = Image.fromarray(overlay)
  output.save(f"{overlay_output_path}", format=f"{overlay_format}")

# Maybe combine both tent counts and return a dictionary with target and estimate values
def gt_tent_count(id):
  with open(TENT_CSV) as csvfile:
    for row in list(csv.reader(csvfile)):
      if row[0].split(".", 1)[0] == id:
        return int(row[1])
  return None

def tent_count(id):
  # TODO: implement this (convolutional neural network here?)
  return None


def get_record(id):
  return int(id.split('_')[1])-1

def save_all(loaders, model, directory=OUTPUT_DIR, device=DEVICE):
  model.eval()
  results = {}
  for dataset, loader in loaders.items():
    dirs = DIRECTORIES[dataset]

    loader_progress = tqdm(loader)
    loader_progress.set_description(f"Saving {dataset} dataset")
    for feature, target, id in loader_progress:
      id = id[0]
      feature = feature.to(device)
      with torch.no_grad():
        estimate = torch.sigmoid(model(feature))
        estimate = (estimate > 0.5).float()

      torchvision.utils.save_image(estimate, f"{dirs['estimates']}{id}.{IMG_FORMAT}")

      # NOTE: The line below was messing up the target; we should check that the target is loading correctly when used by the model to verify the problem is now fixed
      # torchvision.utils.save_image(target.unsqueeze(1), f"{dirs['targets']}{id}.{IMG_FORMAT}") #might not need unsqueeze? or maybe we should use it on estimate too?

      accuracy = similarity(estimate.detach().cpu(), target).item()

      # TODO: store transformation in here too if possible
      # TODO: perhaps results should follow the form of the raw dataset, and maybe just update it?
      # TODO: results should probably store paths as well, but storing just IDs will do for now
      results[id] = {
        "record":          get_record(id),
        "tile_coordinates":find_square_coordinates(get_record(id)),
        "dataset":         dataset,
        "target_count":    gt_tent_count(id),
        "estimate_count":  tent_count(id),
        "mask_similarity": accuracy
      }

      loader_progress.set_description(f"Saving [{dataset}]: {id}, {results[id]}")
    
  model.train()
  with open(RESULT_PICKLE, "wb") as output:
    pickle.dump(results, output, pickle.HIGHEST_PROTOCOL)

  # with open(RESULT_CSV, "w") as output:
  #   writer = csv.DictWriter(output, fieldnames=results.keys())
  #   writer.writeheader()
  #   writer.writerows(results)

  # TODO: use this instead of the above commented out code (adapt the code to make it work, it comes from https://stackoverflow.com/questions/56018692/converting-pkl-file-to-csv-file)
  # with open(RESULT_PICKLE, "rb") as input:
  #     object = pkl.load(input)
  # df = pd.DataFrame(object)
  # df.T.to_csv(r'results.csv')

  shutil.make_archive("tent_counts", "tar", directory)
  print("Saved.\n")

def tile_images(output_name, tile_dir, max_records=256):
  print(f"Beginning Image Tiling ({tile_dir})...")
  square_length = int(math.sqrt(max_records))
  tile_paths = [[None for x in range(square_length)] for y in range(square_length)]
  
  for dirs in DIRECTORIES.values():
    for path, _, tile_names in os.walk(dirs[tile_dir]):
      for tile_name in tile_names:
        tile_index = find_square_coordinates(int(tile_name.split('_')[1].split('.')[0])-1)
        tile_paths[tile_index[0]][tile_index[1]] = (path+tile_name)
  
  if any(tile_paths[0]):
    row_loop = tqdm(range(len(tile_paths)))
    img = None
    w = h = 0
    for r in row_loop:                      # Rows
      for c in range(len(tile_paths[0])):   # Columns
        tile = None
        with Image.open(tile_paths[r][c]) as img_tile:
          tile = img_tile.convert("RGB")

        if img == None:
          img = tile
        if r == 0:
          w += tile.width
        if c == 0:
          h += tile.height

        dst = Image.new('RGB', (w, h))
        dst.paste(img, (0, 0))
        dst.paste(tile, (c*tile.width, r*tile.height))
        img = dst
        img.save(output_name)

        row_loop.set_description(f"W:{w}, H:{h}, Index: ({r}, {c}), Input: {tile_paths[r][c]}, Output: {output_name}")
    print(f"Finished Tiling, Output: {output_name}")
  else:
    raise Exception(f"Selected directory '{tile_dir}' is empty!")

# CNN Counting

In [ ]:
# From https://github.com/Thundertung/Book-Price-regression-CNNs/blob/main/Judging%20a%20book%20by%20its%20cover.ipynb
# and from https://www.youtube.com/watch?v=nU_T2PPigUQ&t=531s
# TODO: adapt for use here

if False: # TODO: set to True if you want to run this
  from sklearn.preprocessing import LabelEncoder    #For encoding categorical variables
  from sklearn.model_selection import train_test_split #For splitting of data
  #All tensorflow utilities for creating, training and working with a CNN
  from tensorflow.keras.utils import to_categorical
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
  from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
  from tensorflow.keras.losses import categorical_crossentropy
  from tensorflow.keras.optimizers import Adam
  from tensorflow.keras.callbacks import ModelCheckpoint
  from tensorflow.keras.models import load_model

  new_train_set = []
  new_train_count = np.empty(len(training_set))
  for i in range(len(training_set)):
    new_train_set.append(np.asarray(training_set[i]['img']))
    new_train_count[i] = training_set[i]['num']

  new_train_set = np.array(new_train_set)

  print(f"Training Set: {np.shape(new_train_set)}")
  print(f"Training Count: {np.shape(new_train_count)}")

  new_val_set = []
  new_val_count = np.empty(len(validation_set))
  for i in range(len(validation_set)):
    new_val_set.append(np.asarray(validation_set[i]['img']))
    new_val_count[i] = validation_set[i]['num']

  new_val_set = np.array(new_val_set)

  print(f"Val Set: {np.shape(new_val_set)}")
  print(f"Val Count: {np.shape(new_val_count)}")

  # new_train_set.reshape(new_train_set.shape[0],new_train_set.shape[1],new_train_set.shape[2],new_train_set.shape[3])
  # new_train_set.reshape(new_train_set.shape[0],new_train_set.shape[1],new_train_set.shape[2],new_train_set.shape[3])


  print(f"Training Set: {np.shape(new_train_set)}")

  # input_shape = np.shape(new_train_set.reshape(np.shape(new_train_set)[1], np.shape(new_train_set)[2]))
  input_shape = np.shape(new_train_set[0])
  print(f"Input Shape: {input_shape}")


  model = Sequential()

  # First conv
  model.add(Conv2D(64, (3,3), activation='relu', input_shape=input_shape))
  model.add(MaxPool2D(2,2))

  # Second conv
  model.add(Conv2D(64,(3,3),activation='relu'))
  model.add(MaxPool2D(2,2))
  model.add(Flatten())

  # Hidden layer
  model.add(Dense(512, activation='relu'))
  model.add(Dense(1, activation='linear'))





  # model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu', input_shape = input_shape))
  # model.add(BatchNormalization())
  # model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
  # model.add(BatchNormalization())
  # # model.add(MaxPool2D(strides=(2,2)))
  # model.add(Dropout(0.25))
  # model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
  # model.add(BatchNormalization())
  # model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
  # model.add(BatchNormalization())
  # # model.add(MaxPool2D(strides=(2,2)))
  # model.add(Dropout(0.25))

  # model.add(Flatten())
  # # model.add(Dense(100, activation='relu'))
  # model.add(Dropout(0.25))

  # # model.add(Dense(1024, activation='relu'))
  # model.add(Dropout(0.4))
  # model.add(Dense(1, activation='linear'))

  learning_rate = 0.001

  model.compile(loss = categorical_crossentropy,
                optimizer = Adam(learning_rate),
                metrics=['accuracy'])

  model.summary()

  # TODO: the sets are an array of dictionaries where each dictionary contains the values we want

  # new_train_set = []
  # new_train_count = np.empty(len(training_set))
  # for i in range(len(training_set)):
  #   new_train_set.append(np.asarray(training_set[i]['img']))
  #   new_train_count[i] = training_set[i]['num']

  # print(np.shape(new_train_set))

  # new_val_set = []
  # new_val_count = np.empty(len(validation_set))
  # for i in range(len(validation_set)):
  #   new_val_set.append(np.asarray(validation_set[i]['img']))
  #   new_val_count[i] = validation_set[i]['num']

  #training_set, validation_set, csv_training_set, csv_validation_set = train_test_split(dataset, csv_dataset, test_size=TEST_SIZE, random_state=RANDOM_STATE)

  model.fit(new_train_set, new_train_count, epochs=15, validation_data=(new_val_set, new_val_count))


  # history = model.fit( X_train, Y_train, 
  #                     epochs = 15, batch_size = 100, 
  #                     callbacks=[save_best2], verbose=1, 
  #                    validation_data = (X_val, Y_price_val))
else:
  print("Skipping for now; change if to True if you want to run this block")

Skipping for now; change if to True if you want to run this block


# Training

In [ ]:
# Model Training
# Adapted From https://www.youtube.com/watch?v=IHq1t7NxS8k



# This whole function trains one epoch
def train_fn(loader, model, optimizer, loss_fn, scaler):
  loop = tqdm(loader) # tqdm gives us a progress bar
  loop.set_description("Training")

  for batch_idx, (data, targets, id) in enumerate(loop):
    loop.set_description(f"Training on {id[0]}")
    data = data.to(device=DEVICE)
    targets = targets.float().unsqueeze(1).to(device=DEVICE) #might not need to make it float since it might already be float? Also, unsqueeze is used cuz we're adding a channel

    # Forward
    with torch.cuda.amp.autocast():
      predictions = model(data)
      loss = loss_fn(predictions, targets)

    # Backward
    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    # Update tqdm loop
    loop.set_postfix(loss=loss.item())

def main():

  # TODO: might need to uncomment the rotations and flips; commented out for now to test why some images are misaligned
  
  train_transform = None
  if DATA_AUGMENTATION:
    train_transform = A.Compose([
      A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
      ###### TODO: Need to undo these transformations when we want to stitch the images
      A.Rotate(limit=35, p=1.0),
      A.HorizontalFlip(p=0.5),
      A.VerticalFlip(p=0.1),
      ######
      A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0 # This basically just divides by 255, so we get a value between 0 and 1
      ),
      ToTensorV2()
    ])
  else:
    train_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    ###### TODO: Need to undo these transformations when we want to stitch the images
    # A.Rotate(limit=35, p=1.0),
    # A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.1),
    ######
    A.Normalize(
      mean=[0.0, 0.0, 0.0],
      std=[1.0, 1.0, 1.0],
      max_pixel_value=255.0 # This basically just divides by 255, so we get a value between 0 and 1
    ),
    ToTensorV2()
  ])

  val_transforms = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.Normalize(
      mean=[0.0, 0.0, 0.0],
      std=[1.0, 1.0, 1.0],
      max_pixel_value=255.0 # This basically just divides by 255, so we get a value between 0 and 1
    ),
    ToTensorV2()
  ])

  model = UNet(in_channels=3, out_channels=1).to(DEVICE) # if we wanted multiclass segmentation we'd change our channels and change our loss function to cross entropy loss
  loss_fn = nn.BCEWithLogitsLoss() # We're not doing sigmoid on the output of model which is why we're using this here
  optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

  train_loader, val_loader = get_loaders(
      TRAIN_IMG_DIR,
      TRAIN_LBL_DIR,
      VAL_IMG_DIR,
      VAL_LBL_DIR,
      BATCH_SIZE,
      train_transform,
      val_transforms,
      NUM_WORKERS,
      PIN_MEMORY
  )

  if os.path.exists(CHECKPOINT):
    print(f"Using device: {DEVICE}")
    if DEVICE == "cuda":
      load_checkpoint(torch.load(CHECKPOINT), model)
    else:
      load_checkpoint(torch.load(CHECKPOINT, map_location=lambda storage, loc: storage), model) # From https://discuss.pytorch.org/t/on-a-cpu-device-how-to-load-checkpoint-saved-on-gpu-device/349/4
  else:
    print("No checkpoint found!\n")

  scaler = torch.cuda.amp.GradScaler() # I think this is where we get the warning about running on CPU; should try to address this warning
  

  if not os.path.exists(OVERVIEWS):
    # TODO: ideally we should be doing this when we initialize all directories instead of here
    os.makedirs(OVERVIEWS)
    os.makedirs(OVERLAYS)
    os.makedirs(ESTIMATE_OVERVIEWS)
    os.makedirs(ESTIMATE_OVERLAYS)
    os.makedirs(TARGET_OVERVIEWS)
    os.makedirs(TARGET_OVERLAYS)
    os.makedirs(ESTIMATE_HEATMAPS)
    os.makedirs(TARGET_HEATMAPS)
  feature_overview_path = f"{OVERVIEWS}features_overview.{IMG_FORMAT}"
  if EPOCH_TILE or TILE:
    gt_overview_path = f"{TARGET_OVERVIEWS}gt_overview.{IMG_FORMAT}"
    gt_overlay_path = f"{TARGET_OVERLAYS}gt_overlay.{IMG_FORMAT}"
    gt_heatmap_path = f"{TARGET_HEATMAPS}gt_heatmap.{IMG_FORMAT}"

    # TODO: make a function for this? since we repeat ourselves here
    if os.path.exists(feature_overview_path):
      print(f"{feature_overview_path} already exists, skipping...")
    else:
      tile_images(feature_overview_path, "features") # Feature tiles
    
    if os.path.exists(gt_overview_path):
      print(f"{gt_overview_path} already exists, skipping...")
    else:
      tile_images(gt_overview_path, "targets") # Ground-truth tiles
    
    if os.path.exists(gt_overlay_path):
      print(f"{gt_overlay_path} already exists, skipping...")
    else:
      save_overlay(feature_overview_path, gt_overview_path, gt_overlay_path)

    if os.path.exists(gt_heatmap_path):
      print(f"{gt_heatmap_path} already exists, skipping...")
    else:
      save_heatmap(gt_overview_path, gt_heatmap_path)
  if SAVE_DATASET_TO_DRIVE and not WORK_IN_DRIVE:
    archive = f'{DRIVE_DIR}data.zip'
    shutil.make_archive(archive.split('.')[0], archive.split('.')[1], 'data')
  epoch_offset = 0
  for epoch in range(NUM_EPOCHS):
    if epoch > 0: print("\n") 
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    train_fn(train_loader, model, optimizer, loss_fn, scaler)

    # Save Model
    checkpoint = {"state_dict":model.state_dict(), "optimizer":optimizer.state_dict()}
    save_checkpoint(checkpoint)
    save_all({"validation":val_loader, "training":train_loader}, model)

    if EPOCH_TILE:
      overview_path = f"{ESTIMATE_OVERVIEWS}overview_{epoch}.{IMG_FORMAT}"
      overlay_path = f"{ESTIMATE_OVERLAYS}overlay_{epoch}.{IMG_FORMAT}"
      heatmap_path = f"{ESTIMATE_HEATMAPS}heatmap_{epoch}.{IMG_FORMAT}"
      while os.path.exists(overview_path): # Will only run on the first pass; this ensures we don't overwrite old overviews
        print(f"Path '{overview_path}' already exists.")
        epoch_offset += 1
        print(f"Offsetting epoch {epoch} by +{epoch_offset}")
        overview_path = f"{ESTIMATE_OVERVIEWS}overview_{epoch+epoch_offset}.{IMG_FORMAT}"
        overlay_path = f"{ESTIMATE_OVERLAYS}overlay_{epoch+epoch_offset}.{IMG_FORMAT}"
        heatmap_path = f"{ESTIMATE_HEATMAPS}heatmap_{epoch+epoch_offset}.{IMG_FORMAT}"
      tile_images(overview_path, "estimates") #"ve_tiles") # Estimate tiles
      save_overlay(feature_overview_path, overview_path, overlay_path)
      save_heatmap(overview_path, heatmap_path)
      if SAVE_OUTPUT_TO_DRIVE and not WORK_IN_DRIVE:
        archive = f'{DRIVE_DIR}epoch_{epoch}.zip'
        shutil.make_archive(archive.split('.')[0], archive.split('.')[1], 'output')
  if TILE and not EPOCH_TILE:
    tile_images(f"{ESTIMATE_OVERVIEWS}overview.{IMG_FORMAT}", "estimates") #"ve_tiles") # Estimate tiles

# Output

In [ ]:
if __name__ == "__main__":
  main()

print("\nDone.")

Using device: cuda
=> Loading checkpoint

output/overviews/features_overview.png already exists, skipping...
output/overviews/targets/gt_overview.png already exists, skipping...
output/overviews/overlays/targets/gt_overlay.png already exists, skipping...

Saving heatmap to 'output/overviews/heatmaps/targets/gt_heatmap.png'...
Epoch 1/50


Training on sarpol_249: 100%|██████████| 179/179 [01:11<00:00,  2.50it/s, loss=0.63]


=> Saving checkpoint



Saving [validation]: sarpol_184, {'record': 183, 'tile_coordinates': (11, 7), 'dataset': 'validation', 'target_count': 154, 'estimate_count': None, 'mask_similarity': 0.8072623014450073}: 100%|██████████| 77/77 [01:16<00:00,  1.01it/s]
Saving [training]: sarpol_141, {'record': 140, 'tile_coordinates': (8, 12), 'dataset': 'training', 'target_count': 0, 'estimate_count': None, 'mask_similarity': 0.02631578966975212}: 100%|██████████| 179/179 [02:43<00:00,  1.10it/s]


Saved.

Path 'output/overviews/estimates/overview_0.png' already exists.
Offsetting epoch 0 by +1
Path 'output/overviews/estimates/overview_1.png' already exists.
Offsetting epoch 0 by +2
Path 'output/overviews/estimates/overview_2.png' already exists.
Offsetting epoch 0 by +3
Path 'output/overviews/estimates/overview_3.png' already exists.
Offsetting epoch 0 by +4
Path 'output/overviews/estimates/overview_4.png' already exists.
Offsetting epoch 0 by +5
Path 'output/overviews/estimates/overview_5.png' already exists.
Offsetting epoch 0 by +6
Path 'output/overviews/estimates/overview_6.png' already exists.
Offsetting epoch 0 by +7
Path 'output/overviews/estimates/overview_7.png' already exists.
Offsetting epoch 0 by +8
Path 'output/overviews/estimates/overview_8.png' already exists.
Offsetting epoch 0 by +9
Path 'output/overviews/estimates/overview_9.png' already exists.
Offsetting epoch 0 by +10
Path 'output/overviews/estimates/overview_10.png' already exists.
Offsetting epoch 0 by +11

...

=> Saving checkpoint



Saving [validation]: sarpol_184, {'record': 183, 'tile_coordinates': (11, 7), 'dataset': 'validation', 'target_count': 154, 'estimate_count': None, 'mask_similarity': 0.6735734939575195}: 100%|██████████| 77/77 [00:08<00:00,  9.59it/s]
Saving [training]: sarpol_071, {'record': 70, 'tile_coordinates': (4, 6), 'dataset': 'training', 'target_count': 6, 'estimate_count': None, 'mask_similarity': 0.9577352404594421}: 100%|██████████| 179/179 [00:18<00:00,  9.66it/s]


Saved.

Path 'output/overviews/estimates/overview_18.png' already exists.
Offsetting epoch 18 by +37
Beginning Image Tiling (estimates)...


W:8192, H:7168, Index: (13, 4), Input: output/training/estimates/sarpol_213.png, Output: output/overviews/estimates/overview_55.png:  81%|████████▏ | 13/16 [04:48<01:06, 22.18s/it]


KeyboardInterrupt: ignored